In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import imutils
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K

In [5]:
pathToExecutable = (
    "C:/Program Files/GNU Octave/Octave-7.1.0/mingw64/bin/octave-cli.exe"
)
os.environ['OCTAVE_EXECUTABLE'] = pathToExecutable
from oct2py import octave

In [4]:
def get_pts(file_pts):
    pts = file_pts.readlines()
    pts = pts[3:len(pts) - 1]
    points = np.zeros((len(pts), 2))
    for i in range(len(pts)):
        pts[i] = pts[i].replace('\n', '')
        x = pts[i].split(' ')
        points[i][0] = float(x[0])
        points[i][1] = float(x[1])

    angle = math.atan2(points[31, 1] - points[36, 1], points[31, 0] - points[36, 0]) * 180 / math.pi
    if angle > 90:
        angle = angle - 180
    if angle < -90:
        angle = angle + 180
    return points, angle

In [13]:
def preprocessing():
    image_path = os.path.join(os.getcwd(),"FGNET","images")
    pts_path = os.path.join(os.getcwd(),"FGNET","points")

    for filename in os.listdir(image_path):
        if(filename.split['.'][1]!=None):
        image = cv2.imread(os.path.join(image_path, filename))
        name_pts = filename.split('.')[0].lower() + ".pts"

        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        file_pts = open(os.path.join(pts_path, name_pts))

        points, angle = get_pts(file_pts)

        rotated = imutils.rotate(image, angle)
        m_i = min(points[16][1], points[22][1])
        croped = rotated[int(m_i - 10):int(points[7][1]), int(points[0][0]):int(points[14][0] + 5)]
        rot_crop_his = cv2.equalizeHist(croped)
        try:
            cv2.imwrite(os.path.join("images_new", filename), rot_crop_his)
        except:
            print(filename)


In [17]:
def get_images_new():
    image_path = os.path.join(os.getcwd(),"images_new")
    labels = np.zeros((1, 1002))
    i = 0
    images = []
    print(labels.shape)
    for filename in os.listdir(image_path):
        if(filename.split('.')[1]!='ipynb_checkpoints'):
            img = load_img(os.path.join(image_path, filename), target_size=(224, 224))
            img = img_to_array(img)
            images.append(img)
            labels[0][i] = int(filename.split('A')[0]) #eg: 001 is the label stored
            i = i + 1
    return np.array(images), labels

In [18]:
def feature_extraction():
    with tf.device('/device:cpu:0'):
        model = tf.keras.applications.VGG16(
            include_top=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None,
            classes=1000,
            classifier_activation="softmax",
        )
    
        images, labels = get_images_new()
        f = open('data.txt', 'w')
        l = len(model.layers)
        for i in range(len(images)):
            image = images[i]
            img = image.reshape((1, image.shape[0], image.shape[1], image.shape[2])) 

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 4).output)
            Flatten = keras_function([img])

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 3).output)
            FC7 = keras_function([img])

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 2).output)
            FC6 = keras_function([img])
        
            f.write(str(labels[0][i]) + '\n')
            np.savetxt(f, Flatten, fmt="%s")
            np.savetxt(f, FC7, fmt="%s")
            np.savetxt(f, FC6, fmt="%s")
        
        f.close()

In [23]:
model = tf.keras.applications.VGG16(
            include_top=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None,
            classes=1000,
            classifier_activation="softmax",
        )
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
print(os.getcwd())
preprocessing()


C:\Users\speci\git_test\Age-invariant-face-recognition-based-on-deep-features-analysis-1
001A02.JPG
001A05.JPG
001A08.JPG
001A10.JPG
001A14.JPG
001A16.JPG
001A18.JPG
001A19.JPG
001A22.JPG
001A28.JPG
001A29.JPG
001A33.JPG
001A40.JPG
001A43a.JPG
001A43b.JPG
002A03.JPG
002A04.JPG
002A05.JPG
002A07.JPG
002A12.JPG
002A15.JPG
002A16.JPG
002A18.JPG
002A20.JPG
002A21.JPG
002A23.JPG
002A26.JPG
002A29.JPG
002A31.JPG
002A36.JPG
002A38.JPG
003A18.JPG
003A20.JPG
003A23.JPG
003A25.JPG
003A35.JPG
003A38.JPG
003A47.JPG
003A49.JPG
003A51.JPG
003A58.JPG
003A60.JPG
003A61.JPG
004A19.JPG
004A21.JPG
004A26.JPG
004A28.JPG
004A30.JPG
004A37.JPG
004A40.JPG
004A48.JPG
004A51.JPG
004A53.JPG
004A62.JPG
004A63.JPG
005A18.JPG
005A24.JPG
005A30.JPG
005A31.JPG
005A35.JPG
005A40.JPG
005A45.JPG
005A48.JPG
005A49.JPG
005A52.JPG
005A61.JPG
006A24.JPG
006A28.JPG
006A31.JPG
006A36.JPG
006A40.JPG
006A42.JPG
006A46.JPG
006A51.JPG
006A55.JPG
006A61.JPG
006A67.JPG
006A69.JPG
007A01.JPG
007A18.JPG
007A22.JPG
007A23.JPG
007A26.

In [19]:
feature_extraction()

(1, 1002)


In [3]:
Flatten = [];
FC7 = [];
FC6 = [];
labels = [];
with open('data.txt') as f:
    lines=f.readlines()
    print(len(lines),lines[0])
    for i in range(0,len(lines),4):
        line = lines[i].replace('\n','')
        line = lines[i].rstrip()
        line=float(line)
        labels.append(line)
        
        fl = lines[i+1].replace('\n','')
        fl = lines[i+1].rstrip()
        fl=fl.split(' ')
        fl = np.array(fl)
        fl = fl.astype(float)
        Flatten.append(fl.tolist())
        
        f7 = lines[i+2].replace('\n','')
        f7 = lines[i+2].rstrip()
        f7=f7.split(' ')
        f7 = np.array(f7)
        f7 = f7.astype(float)
        FC7.append(f7.tolist())
        
        f6 = lines[i+3].replace('\n','')
        f6 = lines[i+3].rstrip()
        f6=f6.split(' ')
        f6 = np.array(f6)
        f6 = f6.astype(float)
        FC6.append(f6.tolist())
        


4008 1.0



In [58]:
print(len(FC7[0]),len(FC6),len(Flatten))

4096 1002 1002


In [57]:
from sklearn.decomposition import PCA
def pcaFuse(layer,n):
    l=np.array(layer)
    pca2 = PCA(n_components=n)
    pca2.fit(l)
    new_l = pca2.transform(l)
    return new_l.tolist()

In [62]:
n=len(Flatten)
flatten_new=pcaFuse(Flatten,n)
fc7_new=pcaFuse(FC7,n)
fc6_new=pcaFuse(FC6,n)

In [77]:
script="""
        function t = fuse(Flatten,FC6,FC7,labels)
            [Ax1,Ay1,Xs1,Ys1] = dcaFuse(Flatten,FC6,labels);
            [Ax2,Ay2,Xs2,Ys2] = dcaFuse(FC7,FC6,labels);
            level2_X = Xs1 + Ys1;
            level2_Y = Xs2 + Ys2;

            [Ax_out,Ay_out,Xs_out,Ys_out] = dcaFuse(level2_X,level2_Y,labels);

            final_output = Xs_out + Ys_out;
            t = final_output';
            
            end
        """
with open("fuse.m","w+") as f:
    f.write(script)
t=octave.fuse(flatten_new,fc7_new,fc6_new,labels)

In [79]:
t.shape

(1002, 81)

In [104]:
with open('testfeatures.txt','w+') as f:
    c=0
    for i in t:
        f.write("%d "%labels[c])
        for j in i:
            f.write("%f "%j)
        f.write("\n")
        c+=1